In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [2]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51268_2024-04-25~2025-04-25.xlsx')
ka_df = pd.read_excel('/Users/seanyu_mac/Downloads/20250425134848.xlsx')

In [3]:
fs_df['옵션명'].unique()

array(['파이토레드큐민(정제)_6박스(36%할인)', '파이토레드큐민(정제)_ 2박스(20%할인)',
       '파이토레드큐민(정제)_4박스(32%할인)', '파이토레드큐민(정제)_12박스(45%할인)',
       '퓨어레드큐민(분말)_2박스(45%할인)', '퓨어레드큐민(분말)_6박스(53%할인)',
       '퓨어레드큐민(분말)_12박스(55%할인)', '퓨어레드큐민(분말)_4박스(49%할인)'], dtype=object)

In [4]:
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: x.split('_')[1].split('박스')[0])
fs_df['박스수량'] = fs_df['박스수량'].astype(int)

In [5]:
fs_df['주문 상태'].unique()

array(['결제완료', '배송준비', '취소완료', '배송중', '배송완료', '판매자취소', '반품진행(수거진행)'],
      dtype=object)

In [6]:
ka_df['박스수량'] = ka_df['옵션'].apply(lambda x: x.split('_')[1].split('박스')[0])
ka_df['박스수량'] = ka_df['박스수량'].astype(int)

In [7]:
fs_df['구분'] = '친한스토어'
ka_df['구분'] = '톡스토어'

In [8]:
fs_df = fs_df[['결제 번호', '주문 상태','결제 완료일', '옵션명', '수량', '박스수량', '정산가', '구분']]

In [9]:
ka_df = ka_df[['결제번호', '주문상태', '발송요청일', '옵션', '수량', '박스수량', '정산기준금액', '구분']]

In [10]:
ka_df.columns = fs_df.columns

In [11]:
df = pd.concat([fs_df, ka_df])

In [12]:
df.columns = ['결제 번호', '주문 상태', '결제 완료일', '옵션명', '수량', '박스수량', '매출', '구분']

In [13]:
df['주문 상태'].unique()

array(['결제완료', '배송준비', '취소완료', '배송중', '배송완료', '판매자취소', '반품진행(수거진행)',
       '301 배송 준비 중', '305 배송 완료', '304 배송 중', '601 구매 결정'], dtype=object)

In [14]:
status = ['결제완료', '배송준비', '배송중', '배송완료', '202 배송 요청', '301 배송 준비 중', '304 배송 중', '305 배송 완료', '601 구매 결정']

In [15]:
df[df['주문 상태'].isin(['취소완료', '판매자취소', '반품진행(반품진행)', '반품진행(수거진행)'])]

,결제 번호,주문 상태,결제 완료일,옵션명,수량,박스수량,매출,구분
7,parma20250425092928DJRJ1,취소완료,2025-04-25 09:29:46.669000,파이토레드큐민(정제)_ 2박스(20%할인),1,2,88000,친한스토어
47,parma20250423104117k7lJ5,판매자취소,2025-04-23 10:41:47.400000,파이토레드큐민(정제)_12박스(45%할인),1,12,363000,친한스토어
53,parma20250423082901n7jP7,반품진행(수거진행),2025-04-23 08:29:43.935000,퓨어레드큐민(분말)_2박스(45%할인),1,2,54000,친한스토어
88,parma202504221511101JrH7,취소완료,2025-04-22 15:11:19.313000,파이토레드큐민(정제)_6박스(36%할인),1,6,211200,친한스토어
119,parma20250422111329VeCd5,취소완료,2025-04-22 11:13:43.406000,파이토레드큐민(정제)_ 2박스(20%할인),1,2,88000,친한스토어
147,parma2025042209364267lD9,반품진행(수거진행),2025-04-22 09:37:54.114000,파이토레드큐민(정제)_12박스(45%할인),1,12,363000,친한스토어


In [16]:
df = df[df['주문 상태'].isin(status)]

In [17]:
df['주문 상태'].unique()

array(['결제완료', '배송준비', '배송중', '배송완료', '301 배송 준비 중', '305 배송 완료',
       '304 배송 중', '601 구매 결정'], dtype=object)

In [18]:
df['결제 완료일'] = pd.to_datetime(df['결제 완료일'])
df['결제 완료일'] = df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
pivot_df = pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['매출', '박스수량'], aggfunc=['sum'], fill_value=0, margins=True, margins_name='총합')
pivot_df.columns = pivot_df.columns.droplevel(0)
temp_df = pd.DataFrame(pivot_df)

pivot = pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합')
temp_df2 = pd.DataFrame(pivot)

result_df = pd.merge(temp_df, temp_df2, left_index=True, right_index=True, how='inner')
result_df.rename(columns={'결제 번호': '주문수량'}, inplace=True)
result_df.to_excel(f'/Users/seanyu_mac/Downloads/샤인약사님_매출_{datetime.now()}.xlsx')
result_df.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     옵션명                                              
2025-04-22 선택: 파이토레드큐민(정제)_ 2박스(20%할인)     176,000    4    2
           선택: 파이토레드큐민(정제)_12박스(45%할인)   1,815,000   60    5
           선택: 파이토레드큐민(정제)_4박스(32%할인)      448,800   12    3
           선택: 퓨어레드큐민(분말)_12박스(55%할인)      267,000   12    1
           선택: 퓨어레드큐민(분말)_2박스(45%할인)       162,000    6    3
           선택: 퓨어레드큐민(분말)_4박스(49%할인)       396,000   16    4
           파이토레드큐민(정제)_ 2박스(20%할인)       2,024,000   46   23
           파이토레드큐민(정제)_12박스(45%할인)       7,260,000  228   19
           파이토레드큐민(정제)_4박스(32%할인)        4,039,200  108   27
           파이토레드큐민(정제)_6박스(36%할인)        3,590,400  102   17
           퓨어레드큐민(분말)_12박스(55%할인)          267,000   12    1
           퓨어레드큐민(분말)_2박스(45%할인)           324,000   12    6
           퓨어레드큐민(분말)_4박스(49%할인)           495,000   20    5
           퓨어레드큐민(분말)_6박스(53%할인)           139,000    6    1
2025-04-23 선택: 파이토레드큐민(정제)_4박스(32%할인)      448,800   12    3
           파이토레드큐민(정제)_ 2박스(20%할인)         528,000   12    6
           파이토레드큐민(정제)_12박스(45%할인)         726,000   24    2
           파이토레드큐민(정제)_4박스(32%할인)          897,600   24    6
           퓨어레드큐민(분말)_2박스(45%할인)           162,000    6    3
           퓨어레드큐민(분말)_4박스(49%할인)           297,000   12    3
           퓨어레드큐민(분말)_6박스(53%할인)           139,000    6    1
2025-04-24 선택: 파이토레드큐민(정제)_4박스(32%할인)      149,600    4    1
           선택: 퓨어레드큐민(분말)_2박스(45%할인)        54,000    2    1
           선택: 퓨어레드큐민(분말)_4박스(49%할인)        99,000    4    1
           파이토레드큐민(정제)_ 2박스(20%할인)         352,000    8    4
           파이토레드큐민(정제)_12박스(45%할인)       1,089,000   36    3
           파이토레드큐민(정제)_4박스(32%할인)          598,400   16    4
           파이토레드큐민(정제)_6박스(36%할인)          633,600   18    3
           퓨어레드큐민(분말)_12박스(55%할인)          267,000   12    1
           퓨어레드큐민(분말)_2박스(45%할인)           162,000    6    3
2025-04-25 파이토레드큐민(정제)_ 2박스(20%할인)         176,000    4    2
           파이토레드큐민(정제)_12박스(45%할인)       1,815,000   60    5
           파이토레드큐민(정제)_4박스(32%할인)          448,800   12    3
           파이토레드큐민(정제)_6박스(36%할인)          422,400   12    2
           퓨어레드큐민(분말)_2박스(45%할인)           108,000    4    2
           퓨어레드큐민(분말)_6박스(53%할인)           139,000    6    1
총합                                      31,115,600  944  177

In [19]:
df['제형'] = df['옵션명'].apply(lambda x: x.split('_')[0].split('(')[1])
df['제형'] = df['제형'].apply(lambda x: x[:2])
temp_df1 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '제형'], values=['매출', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='총합'))
temp_df2 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '제형'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합'))
temp_df2.columns = ['주문수량']

result = pd.merge(temp_df1, temp_df2, left_index=True, right_index=True, how='inner')
result.to_excel(f'/Users/seanyu_mac/Downloads/샤인약사님_매출_{datetime.now()}.xlsx')
result.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     제형                      
2025-04-22 분말   2,050,000   84   21
           정제  19,353,400  560   96
2025-04-23 분말     598,000   24    7
           정제   2,600,400   72   17
2025-04-24 분말     582,000   24    6
           정제   2,822,600   82   15
2025-04-25 분말     247,000   10    3
           정제   2,862,200   88   12
총합             31,115,600  944  177

In [54]:
temp_df3 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values=['매출', '박스수량'], aggfunc='sum'))
temp = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values='결제 번호', aggfunc='count'))
temp.rename(columns={'결제 번호': '주문수량'}, inplace=True)
col = temp['주문수량']
temp_df3['주문수량'] = col
temp_df3.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     구분                         
2025-04-22 친한스토어  18,138,600  534   99
           톡스토어    3,264,800  110   18
2025-04-23 친한스토어   2,749,600   84   21
           톡스토어      448,800   12    3
2025-04-24 친한스토어   3,102,000   96   18
           톡스토어      302,600   10    3
2025-04-25 친한스토어   3,109,200   98   15